In [31]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score

pd.set_option('display.max_rows', 200)

In [32]:
ratings = pd.read_csv("../input/the-movies-dataset/ratings_small.csv")
movie = pd.read_csv("../input/the-movies-dataset/movies_metadata.csv")

#Definition du tableau des notes
ratings = ratings.drop(["timestamp"],axis = 1)
ratings = ratings.pivot(index = 'userId', columns = 'movieId', values = 'rating')
ratings = ratings.fillna(0)

#Definition du tableau de similarité
ratings_std = ratings.apply(standarize)

item_similarity = cosine_similarity(ratings_std.T)

item_similarity_df = pd.DataFrame(item_similarity,index=ratings.columns,columns=ratings.columns)


In [33]:
#Standardize les lignes
def standarize(row):
        new_row = (row - row.mean()) /(row.max() - row.min())
        return new_row

#Retourne le tableau de similarité de film
def get_similar_movies(movie_name,rating):
    similar_score = item_similarity_df[movie_name]*(rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    similar_score = similar_score.drop([movie_name])
    similar_score[similar_score<0]=0
    return similar_score

#Le tableau des films et les notes apporté par l'utilisateur
def tabUserRating(ratings, user_id, id_film_supr):
    ratings2 = ratings
    ratings2 = ratings2[ratings2['userId'] == user_id]
    ratings2 = ratings2.drop(["timestamp"],axis = 1)
    ratings2 = ratings2[ratings2.movieId != id_film_supr]
    return ratings2

#Meme fonction que tabUserRating mais sans suprimer un film
def tabUserRatingSansSupr(ratings, user_id):
    ratings2 = ratings
    ratings2 = ratings2[ratings2['userId'] == user_id]
    ratings2 = ratings2.drop(["timestamp"],axis = 1)
    return ratings2

#Fonction qui retourne la moyenne des similarité entre les films d'un tableau de film en fonction de leurs similarité et de la note attribué
def add_movies(tab_sim,movie_name,rating):
    X = get_similar_movies(movie_name,rating)
    Y = pd.concat([tab_sim,X], axis=1)
    if 'average' in Y:
        Y = Y.drop(['average'],axis=1)
    Y['average'] = Y.mean(axis=1)
    Y = Y.sort_values(ascending=False, by=["average"])
    return Y

#Fonction qui dresse le tableau des similarité et des notes entre les films et retrouve le film supprimer de la liste des films de l'utilisateur
def similarityTabUserRating(tabUserRating, id_film_supr):
    s = pd.DataFrame()
    for i,row in tabUserRating.iterrows():
        s = add_movies(s,row["movieId"],row['rating'])
    for id_film in tabUserRating['movieId']:
        s = s.drop([id_film,])        
    s['Verif'] = np.where(s.index==id_film_supr,1 , 0)    
    return s

#Fonction qui retourne la moyenne des similarité entre les films d'un tableau de film en fonction de leurs similarité
def add_m(tab_sim,movie_name):
    similar_score = item_similarity_df[movie_name]+1
    similar_score = similar_score.drop([movie_name])
    Y = pd.concat([tab_sim,similar_score], axis=1)
    if 'average' in Y:
        Y = Y.drop(['average'],axis=1)
    Y['average'] = Y.mean(axis=1)
    Y = Y.sort_values(ascending=False, by=["average"])
    return Y

#Fonction qui retourne le tableau d'un utilisateur
def tabUser(ratings, user_id, id_film_supr):
    ratings2 = ratings
    ratings2 = ratings2[ratings2['userId'] == user_id]
    ratings2 = ratings2.drop(["timestamp","rating"],axis = 1)
    ratings2 = ratings2[ratings2.movieId != id_film_supr]
    return ratings2

#Meme fonction que tabUser mais sans suprimer un film
def tabUserSansSupr(ratings, user_id):
    ratings2 = ratings
    ratings2 = ratings2[ratings2['userId'] == user_id]
    ratings2 = ratings2.drop(["timestamp","rating"],axis = 1)
    return ratings2

#Fonction qui dresse le tableau des similarité entre les films et retrouve le film supprimer de la liste des films de l'utilisateur
def similarityTabUser(tabUser, id_film_supr):
    s = pd.DataFrame()
    tabUser= tabUser['movieId']
    for id_film in tabUser:
        s = add_m(s,id_film)
    for id_film in tabUser: 
        s = s.drop([id_film])
    s['Verif'] = np.where(s.index==id_film_supr,1 , 0)
    return s

#Meme fonction que similarityTabUser mais sans suprimer un film
def similarityTabUserSansSupr(tabUser):
    s = pd.DataFrame()
    tabUser= tabUser['movieId']
    for id_film in tabUser:
        s = add_m(s,id_film)
    for id_film in tabUser: 
        s = s.drop([id_film])
    return s

#Meme fonction que similarityTabUserRating mais sans suprimer un film
def similarityTabUserRatingSansSupr(tabUserRating):
    s = pd.DataFrame()
    for i,row in tabUserRating.iterrows():
        s = add_movies(s,row["movieId"],row['rating'])
    for id_film in tabUserRating['movieId']:
        s = s.drop([id_film,])          
    return s


In [34]:
#Fonction qui donne la moyenne des Roc au score pour la version ratings
def rocGlobalRatings(user_id,ratingsvierge):
    tabRoc = pd.DataFrame()
    tabUserR = tabUserRatingSansSupr(ratingsvierge,user_id)
    tabUserR = tabUserR['movieId']
    for x in tabUserR:
        tabUserRatings = tabUserRating(ratingsvierge, user_id , x)
        s = similarityTabUserRating(tabUserRatings, x)
        a = roc_auc_score(s['Verif'],s['average'], average=None)
        tabRoc = tabRoc.append({"roc" : a},ignore_index=True)
        tabRoc.to_csv('tabRocRatings.csv',sep=",",index=False,header=False)
        tabRocMean = tabRoc["roc"].mean()
    return tabRocMean

ratingsvierge = pd.read_csv("../input/the-movies-dataset/ratings_small.csv")
user = 1

rocGlobalRatings(user,ratingsvierge) 

In [35]:
#Fonction qui donne la moyenne des Roc au score pour la version classique
def rocGlobal(user_id,ratingsvierge):
    tabRoc = pd.DataFrame()
    tabUserR = tabUserSansSupr(ratingsvierge,user_id)
    tabUserR = tabUserR['movieId']
    for x in tabUserR:
        tabUsers = tabUser(ratingsvierge, user_id , x)
        s = similarityTabUser(tabUsers, x)
        a = roc_auc_score(s['Verif'],s['average'], average=None)
        tabRoc = tabRoc.append({"roc" : a},ignore_index=True)
        tabRoc.to_csv('tabRoc.csv',sep=",",index=False,header=False)
        tabRocMean = tabRoc["roc"].mean()
    return tabRocMean

ratingsvierge2 = pd.read_csv("../input/the-movies-dataset/ratings_small.csv")
user = 1

rocGlobal(user,ratingsvierge2) 

In [36]:
#Fonction qui donne le top 100 des films recommandés
def top100(user_id,ratingsvierge):
    ratings = ratingsvierge
    ratings = ratings[ratings['userId'] == user_id]
    ratings = ratings.drop(["timestamp","rating"],axis = 1)
    tab100 = similarityTabUserSansSupr(ratings)
    tab100['index'] = tab100.index
    tab100 = tab100['index']
    tab100 = tab100.head(100)
    tab100.to_csv('top100.csv',sep=",",index=False,header=False)
    return tab100

ratingsvierge3 = pd.read_csv("../input/the-movies-dataset/ratings_small.csv")
user = 1

top100(user,ratingsvierge3)

In [37]:
#Fonction qui donne le top 100 des films recommandés en prenant en compte les notes données par l'utilisateur
def top100ratings(user_id,ratingsvierge):
    ratings = ratingsvierge
    ratings = ratings[ratings['userId'] == user_id]
    ratings = ratings.drop(["timestamp"],axis = 1)
    tab100 = similarityTabUserRatingSansSupr(ratings)
    tab100['index'] = tab100.index
    tab100 = tab100['index']
    tab100 = tab100.head(100)
    tab100.to_csv('top100ratings.csv',sep=",",index=False,header=False)
    return tab100

ratingsvierge4 = pd.read_csv("../input/the-movies-dataset/ratings_small.csv")
user = 1

top100ratings(user,ratingsvierge4)